In [8]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

In [15]:
def transitions(x,o,iw):
    """for a given essay, return the probability shifts for each word
    this has been adjusted to just return probabilities sequentially,
    instead of adding all probabilities from the same word"""
    worddeltas=[]
    endpoint = o.shape[0]
    startpoint = endpoint-np.count_nonzero(x)
    initval=o[startpoint-1]
    words = []
    probs = np.empty((0,4))
    for i in range(startpoint,endpoint):
        deltacuzofx=np.subtract(o[i],initval)
        words.append(iw[x[i]])
        probs = np.append(probs, deltacuzofx.reshape(1,4), axis=0)
        #if word in worddeltas:
        #    worddeltas[word]=worddeltas[word]+deltacuzofx
        #else:
        #    worddeltas[word]=deltacuzofx
        initval=o[i]
    return (words, probs)

In [16]:
def addtransitions(x,o,iw):
    """for each essay, get the transitions associated with each word
    return a dictionary of word transitions summed across the corpus"""
    addworddeltas={}
    for i in range(x.shape[0]):
        wd=transitions(x[i],o[i],iw)
        for key, value in wd.iteritems():
            if key in addworddeltas:
                addworddeltas[key]=addworddeltas[key]+value
            else:
                addworddeltas[key]=value
    return addworddeltas

In [4]:
def filtertestdata(x,y,o,t,pos):
    t=np.asarray(t)
    ynew=y[(y[:,pos]==1)]
    xnew=x[(y[:,pos]==1)]
    onew=o[(y[:,pos]==1)]
    tnew=t[(y[:,pos]==1)]
    return xnew,ynew,onew,tnew.tolist()

In [5]:
def filterdictionary(d,pos):
    newdic={}
    values=[]
    words=[]
    for key, value in d.iteritems():
        words.append(key)
        values.append(value[pos])
    words=[x for (y,x) in sorted(zip(values,words))]
    values=[y for (y,x) in sorted(zip(values,words))]
    newdic = dict(zip(words, values))
    return words,values,newdic
    

In [2]:
# main function - Values(-1-ALL, 0-control male, 1-control female, 2-affirmation male, 3-affirmation female)
from __future__ import division
index_word=np.load('Data/finalgenderdict.npy').item()
word_index=np.load('Data/finalgenderdictinv.npy').item()
testdata_output=np.load('Data/finalgendertestoutput.npy')
testdata_x=np.load('Data/finalgenderxtestdata.npy')
testdata_y=np.load('Data/finalgenderytestdata.npy')
with open('Data/finalgendertextsinput.txt') as f:
    text_content = f.readlines()
filtertestdatavalue=-1
filterdictionaryvalue=1
if filtertestdatavalue!=-1:
    testdata_x,testdata_y,testdata_output,text_content=filtertestdata(testdata_x,testdata_y,testdata_output,text_content,filtertestdatavalue)
maxindexoutput=np.argmax(testdata_output[:,-1,:],axis=1)
maxindexyval=np.argmax(testdata_y,axis=1)
accuracy=(np.count_nonzero(maxindexoutput == maxindexyval)/maxindexyval.shape[0])*100
#addwd=addtransitions(testdata_x,testdata_y,testdata_output,index_word)
#words,values,dic=filterdictionary(addwd,filterdictionaryvalue)
accuracy

58.8

In [68]:
index_word = np.load('Gender/testsentiw.npy').item()
word_index = np.load('Gender/testsentiw_inv.npy').item()
preds = np.load('Gender/testsentpreds.npy')
x_dat = np.load('Gender/testsentxdat.npy')
df = np.load('Gender/testdat.npy')

index_word_wo = np.load('Gender/testsentiw_wo.npy').item()
word_index_wo = np.load('Gender/testsentiw_inv_wo.npy').item()
preds_wo = np.load('Gender/testsentpreds_wo.npy')
x_dat_wo = np.load('Gender/testsentxdat_wo.npy')

In [69]:
df = pd.DataFrame(df, columns=['word', 'pos', 'lemma', 'sent', 'test_sent', 'sent_wo', 'test_sent_wo'])


- index_word is pairing of index numbers with words
- word_index is the pairing of words with index numbers
- testdata_output is the sequential probabilities for each of the 500 test essays
- testdata_x is a matrix of index_word values (i.e. tells you what word is in the essay
- testdata_y is a matrix that indicates the class for each essay
- accuracy is not scored in the same manner as essay classification (f1 score)

First, 

In [70]:
words=[]
essay=[]
probs = np.empty((0,4))
pos = []
for i in range(0, len(x_dat)):
    word_dists = transitions(x_dat[i],preds[i],index_word)
    words.append(word_dists[0])
    essay.append([i]*len(word_dists[0]))
    pos.append([df.pos.loc[i]]*len(word_dists[0]))
    probs = np.append(probs, word_dists[1], axis=0)
df1 = pd.DataFrame({'word': [w for e in words for w in e],
                    'prob': probs[:,0],
                    'class': ['control_m']*len(probs),
                    'sentence_num': [w for e in essay for w in e],
                    'pos': [p for w in pos for p in w]})
df2 = pd.DataFrame({'word': [w for e in words for w in e],
                    'prob': probs[:,1],
                    'class': ['control_f']*len(probs),
                    'sentence_num': [w for e in essay for w in e],
                    'pos': [p for w in pos for p in w]})
df3 = pd.DataFrame({'word': [w for e in words for w in e],
                    'prob': probs[:,2],
                    'class': ['aff_m']*len(probs),
                    'sentence_num': [w for e in essay for w in e],
                    'pos': [p for w in pos for p in w]})
df4 = pd.DataFrame({'word': [w for e in words for w in e],
                    'prob': probs[:,3],
                    'class': ['aff_f']*len(probs),
                    'sentence_num': [w for e in essay for w in e],
                    'pos': [p for w in pos for p in w]})
df_w = pd.concat([df1,df2,df3,df4])
#df.to_csv('output/nn_probs.csv', index=False, encoding='utf-8')

In [71]:
words=[]
essay=[]
probs = np.empty((0,4))
pos = []
for i in range(0, len(x_dat)):
    word_dists = transitions(x_dat_wo[i],preds_wo[i],index_word_wo)
    words.append(word_dists[0])
    essay.append([i]*len(word_dists[0]))
    pos.append([df.pos.loc[i]]*len(word_dists[0]))
    probs = np.append(probs, word_dists[1], axis=0)
df1 = pd.DataFrame({'word': [w for e in words for w in e],
                    'prob': probs[:,0],
                    'class': ['control_m']*len(probs),
                    'sentence_num': [w for e in essay for w in e],
                    'pos': [p for w in pos for p in w]})
df2 = pd.DataFrame({'word': [w for e in words for w in e],
                    'prob': probs[:,1],
                    'class': ['control_f']*len(probs),
                    'sentence_num': [w for e in essay for w in e],
                    'pos': [p for w in pos for p in w]})
df3 = pd.DataFrame({'word': [w for e in words for w in e],
                    'prob': probs[:,2],
                    'class': ['aff_m']*len(probs),
                    'sentence_num': [w for e in essay for w in e],
                    'pos': [p for w in pos for p in w]})
df4 = pd.DataFrame({'word': [w for e in words for w in e],
                    'prob': probs[:,3],
                    'class': ['aff_f']*len(probs),
                    'sentence_num': [w for e in essay for w in e],
                    'pos': [p for w in pos for p in w]})
df_wo = pd.concat([df1,df2,df3,df4])

In [72]:
df_wo['cond'] = 'without'
df_w['cond'] = 'with'
outdat = pd.concat([df_wo, df_w])
outdat.to_csv('w_wo_nnprobs.csv', index=False, encoding='utf-8')

In [62]:
outdat['class'].value_counts()

aff_f        2922
aff_m        2922
control_m    2922
control_f    2922
Name: class, dtype: int64

In [45]:
outdat.shape

(15840, 6)